In [2]:
import requests
import pandas as pd
import re
from sqlalchemy import create_engine, inspect, text
import matplotlib.pyplot as plt
from abc import abstractmethod
from itertools import combinations
from dataclasses import dataclass
from backtesting import Backtest, Strategy
import numpy as np
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
import seaborn as sns
from dotenv import load_dotenv
import os
load_dotenv()
api_key = os.getenv('api_key')

##### Custom Order Management


In [2]:
class OrderManagementSystem:
    def __init__(self,slippage=0.01, commission=0.001, constraints=None):
        #self.orders is an array of type orders classes)
        self.orders = []
        self.changeConstraints(constraints)
        self.slippage = slippage  # Percentage slippage on each order
        self.commission = commission  # Percentage commission on each order
        

    
    def changeConstraints(self, constraints):
        """
        Change the constraints of the order management system.
        """
        self.constraints = constraints
        self.stopLoss = constraints['stopLoss']
        self.takeProfit = constraints['takeProfit']
        self.shortOnly = constraints['shortOnly']
        self.longOnly = constraints['longOnly']
        self.holdingPeriod = constraints['holdingPeriod']
        print(f"✅ Constraints updated: {self.constraints}")
        
    def initWorkingData(self, columns):
        """
        Initialize the working data DataFrame with specified columns.
        Working data is used to store the results of the strategy execution
        and historical ticker data
        """
        self.workingData = pd.DataFrame(columns=columns)
        print("✅ Working data initialized with columns:", columns)
        
    def addRow (self, row):
        """
        Add a row to the working data DataFrame.
        """
        self.workingData = pd.concat([self.workingData, pd.DataFrame([row])], ignore_index=True)
        self.checkOpenOrders()
        
    def shouldLiquidate(self, order):
        """
        Check if an order should be liquidated based on stop loss or take profit conditions.
        """
        last_row = self.workingData.iloc[-1]
        ticker = order['ticker']
        
        if order['stopLoss'] is not None and last_row[f'{ticker}_close'] <= order['stopLossPrice']:
            print(f"❗ Liquidating {order['signal']} order for {ticker} due to stop loss.")
            self.liquidateOrder(order)
            return True
        
        if order['takeProfit'] is not None and last_row[f'{ticker}_close'] >= order['takeProfitPrice']:
            print(f"❗ Liquidating {order['signal']} order for {ticker} due to take profit.")
            self.liquidateOrder(order)
            return True
        
        return False
    
    def liquidateOrder(self, order):
        """
        Liquidate an order by removing it from the open orders list.
        This method can be called when a stop loss or take profit condition is met.
        Need to implement the logic to remove the order from the orders list.
        Need to implement the logic to update the working data with the liquidation details.
        Need to implement the logic to reverse order position in the working data.
        """
        print(f"✅ Liquidated order: {order}")
        
        # Update working data with liquidation details
        last_row = self.workingData.iloc[-1]
        ticker = order['ticker']
        orderPosition = order['position']
        # Adjust the position in workingData to reflect liquidation
        # Find the current position for the ticker in the last row
        current_position = last_row.get(f'{ticker}_position', 0)
        # Set the new position to zero (fully liquidated)
        newPosition = current_position - orderPosition +self.slippage + self.commission
        
        #update the working data with the new position
        self.workingData.at[last_row.name, f'{ticker}_position'] = newPosition
            
        
    def checkOpenOrders(self):
        """
        Check if there are any open orders that need to be executed.
        This method can be called after adding a new row to the working data.
        """
        if not self.orders:
            print("✅ No open orders to check.")
            return
        
        for order in self.orders:
            #a simple order is when stopLoss and takeProfit are None
            if order['stopLoss'] is None and order['takeProfit'] is None or order['expired']:
                continue
            
            self.shouldLiquidate(order)
            order['expired'] = True
        
    def addOrder(self, signal):
        finalOrder = self.ExecuteOrder(signal)
        if finalOrder is not None:
            self.orders.append(finalOrder)
        
    def calculateRollingMetrics(self, idx, ticker, position, return_of_stock, return_of_strategy, complex=False):
        if complex:
            self.workingData.at[idx, f'{ticker}_position'] = position
            self.workingData.at[idx, f'{ticker}_return_of_stock'] = return_of_stock
            self.workingData.at[idx, f'{ticker}_return_of_strategy'] = return_of_strategy
        else:
            self.workingData.at[idx, f'{ticker}_position'] = position
            self.workingData.at[idx, f'{ticker}_return_of_stock'] = return_of_stock
            self.workingData.at[idx, f'{ticker}_return_of_strategy'] = return_of_strategy
            
        # Add rolling metrics
        window = 20 
        
        if len(self.workingData) >= window:
            recent = self.workingData.iloc[-window:]
            sharpe = (
                recent[f'{ticker}_return_of_strategy'].mean() /
                recent[f'{ticker}_return_of_strategy'].std()
                if recent[f'{ticker}_return_of_strategy'].std() != 0 else np.nan
            )
            volatility = recent[f'{ticker}_return_of_strategy'].std()
            cov = recent[[f'{ticker}_return_of_strategy', f'{ticker}_return_of_stock']].cov().iloc[0, 1]
            var_stock = recent[f'{ticker}_return_of_stock'].var()
            beta = cov / var_stock if var_stock != 0 else np.nan

            self.workingData.at[idx, f'{ticker}_rolling_sharpe'] = sharpe
            self.workingData.at[idx, f'{ticker}_rolling_volatility'] = volatility
            self.workingData.at[idx, f'{ticker}_rolling_beta'] = beta
    
        
            
    def executeOrder(self, Signal):
        signal = Signal['signal']
        ticker = Signal['ticker']
        
        if signal != 'close':
            #checking for long or short only constraints
            if self.longOnly and signal == 'sell':
                print("❌ Long only constraint violated. Cannot execute sell order.")
                return None
            if self.shortOnly and signal == 'buy':
                print("❌ Short only constraint violated. Cannot execute buy order.")
                return None
        
        
        last_row = self.workingData.iloc[-1]
        confidence = Signal.get('confidence', 1)
        if signal== 'close':
            confidence = 1
        position = ((confidence * (1 if signal == 'buy' else -1)) * (1-(self.slippage+self.commission))) if signal != 'close' else 0
        
        return_of_stock = last_row[f'{ticker}_close'] / last_row[f'{ticker}_open'] - 1
        return_of_strategy = return_of_stock * position

        idx = last_row.name
        
        self.calculateRollingMetrics(idx, ticker,position,  return_of_stock, return_of_strategy)
        if self.stopLoss is not None:
            stopLossPrice = last_row[f'{ticker}_open'] * (1 - self.stopLoss) if signal == 'buy' else last_row[f'{ticker}_open'] * (1 + self.stopLoss)
            expired = False
        else:
            stopLossPrice = None
            expired = True
        if self.takeProfit is not None:
            takeProfitPrice = last_row[f'{ticker}_open'] * (1 + self.takeProfit) if signal == 'buy' else last_row[f'{ticker}_open'] * (1 - self.takeProfit)
            expired = False
        else:
            takeProfitPrice = None
            expired = True
        order = {
            'ticker': ticker,
            'signal': signal,
            'position': position,
            'confidence': confidence,
            'idx': idx,
            'stopLoss': self.stopLoss,
            'takeProfit': self.takeProfit,
            'stopLossPrice': stopLossPrice,
            'takeProfitPrice': takeProfitPrice,
            'expired': expired,
        }
        return order
        
            
        
    def ExecuteOrder(self, signal):
        order = self.executeOrder(signal)
        if order is not None:
            return order
        else:
            return None

##### Database


In [ ]:
class PostgresManager:
    def __init__(self, host, port, dbname, user, password):
        self.db_params = {
            'host': host,
            'port': port,
            'dbname': dbname,
            'user': user,
            'password': password
        }
        self.engine_str = (
            f"postgresql://{user}:{password}@{host}:{port}/{dbname}"
        )
        self.engine = create_engine(self.engine_str)
        print("✅ PostgreSQL connection initialized.")

    def upload_dataframe(self, df: pd.DataFrame, table_name: str, if_exists='replace'):
        """
        Uploads a DataFrame to PostgreSQL.
        - if_exists: 'replace', 'append', or 'fail'
        """
        try:
            df.to_sql(table_name, self.engine, if_exists=if_exists, index=False, method='multi')
            print(f"✅ Data uploaded to table '{table_name}'.")
        except Exception as e:
            print(f"❌ Failed to upload to '{table_name}': {e}")
    
    def getTicker30MinData(self, ticker: str) -> pd.DataFrame:
        """
        Retrieves 30-minute interval data for a given ticker from PostgreSQL
        """
        table_name = f"{ticker.upper()}_30MinData"
        try:
            query = f"SELECT * FROM \"{table_name}\""
            df = pd.read_sql_query(query, self.engine)
            
            # Convert 'date' column to datetime and set as index
            if 'date' in df.columns:
                df['date'] = pd.to_datetime(df['date'])
                df.set_index('date', inplace=True)
                df.sort_index(inplace=True)
                print(f"✅ Retrieved {len(df)} rows of 30-min data for {ticker}")
            else:
                print(f"❌ 'date' column missing in table {table_name}")
            return df
        except Exception as e:
            print(f"❌ Failed to fetch data for {ticker}: {e}")
            return pd.DataFrame()
    def getTickerEODData(self, ticker: str) -> pd.DataFrame:
        """
        Retrieves EOD data for a given ticker from PostgreSQL
        """
        table_name = f"{ticker.upper()}_EOD_Data"
        try:
            query = f"SELECT * FROM \"{table_name}\""
            df = pd.read_sql_query(query, self.engine)
            
            # Convert 'date' column to datetime and set as index
            if 'date' in df.columns:
                df['date'] = pd.to_datetime(df['date'])
                df.set_index('date', inplace=True)
                df.sort_index(inplace=True)
                print(f"✅ Retrieved {len(df)} rows of EOD data for {ticker}")
            else:
                print(f"❌ 'date' column missing in table {table_name}")
            return df
        except Exception as e:
            print(f"❌ Failed to fetch data for {ticker}: {e}")
            return pd.DataFrame()
        
    def get_tickers_from_30min_tables(self):
        """
        Extracts all tickers from tables that match the {ticker}_30MinData format.
        """
        try:
            inspector = inspect(self.engine)
            all_tables = inspector.get_table_names()
            pattern = re.compile(r'^(.*)_30MinData$', re.IGNORECASE)
            tickers = [match.group(1).upper() for table in all_tables if (match := pattern.match(table))]
            return tickers
        except Exception as e:
            print(f"❌ Failed to inspect tables: {e}")
            return []
    def get_tickers_from_EOD_tables(self):
        """
        Extracts all tickers from tables that match the {ticker}_30MinData format.
        """
        try:
            inspector = inspect(self.engine)
            all_tables = inspector.get_table_names()
            pattern = re.compile(r'^(.*)_EOD_Data$', re.IGNORECASE)
            tickers = [match.group(1).upper() for table in all_tables if (match := pattern.match(table))]
            return tickers
        except Exception as e:
            print(f"❌ Failed to inspect tables: {e}")
            return []
        
        
#example usage
pg = PostgresManager(
    host=os.getenv('host'),
    port=os.getenv('port'),
    dbname=os.getenv('dbname'),
    user=os.getenv('user'),
    password=os.getenv('password')
)
# Get all tickers from 30-min tables
tickers = pg.get_tickers_from_30min_tables()
print("Tickers found:", tickers)

# Fetch 30-min data for a specific ticker
ticker_data = pg.getTicker30MinData('AAPL')
print(ticker_data.head())

✅ PostgreSQL connection initialized.
Tickers found: ['AVGO', 'MA', 'JNJ', 'PM', 'IBM', 'NOW', 'TBB', 'RTX', 'ADBE', 'SCHW', 'TMO', 'C', 'GILD', 'PANW', 'CMCSA', 'MU', 'APH', 'BX', 'ICE', 'WELL', 'AMT', 'BMYMP', 'SOJC', 'CEG', 'HCA', 'DUKB', 'IBKR', 'ELV', 'INTC', 'AJG', 'TDG', 'CVS', 'RSG', 'ORLY', 'MMM', 'DELL', 'SCCO', 'APO', 'ZTS', 'ECL', 'SNPS', 'RCL', 'BRK-B', 'NFLX', 'HD', 'TMUS', 'WFC', 'AXP', 'ISRG', 'VZ', 'TXN', 'SPGI', 'BSX', 'TJX', 'RCIT', 'PFE', 'COF', 'CRWD', 'LMT', 'COP', 'MSFT', 'BRK-A', 'ORCL', 'BAC', 'UNH', 'CVX', 'NVDA', 'TSLA', 'COST', 'ABBV', 'GE', 'ABT', 'MS', 'MRK', 'BKNG', 'CAT', 'BA', 'NEE', 'APP', 'AMAT', 'ADP', 'LOW', 'VRTX', 'LRCX', 'KKR', 'MSTR', 'PLD', 'CME', 'SOJE', 'SOJD', 'NKE', 'FI', 'SHW', 'EQIX', 'ABNB', 'PH', 'CI', 'CDNS', 'FTNT', 'AAPL', 'WMT', 'XOM', 'KO', 'CRM', 'MCD', 'DIS', 'GS', 'UBER', 'QCOM', 'AMGN', 'HON', 'DE', 'UNP', 'GEV', 'ANET', 'MMC', 'ADI', 'KLAC', 'SBUX', 'MO', 'BMY', 'SO', 'WM', 'DASH', 'CTAS', 'DUK', 'MCK', 'MCO', 'MDLZ', 'UPS', 'A

##### Getting all EOD Data


In [4]:
# 1. Get valid symbols from CompanyProfiles
query = """
SELECT symbol FROM "CompanyProfiles"
WHERE "marketCap" IS NOT NULL AND "marketCap" > 0
"""
symbols_df = pd.read_sql_query(text(query), pg.engine)
tickers = symbols_df['symbol'].tolist()

print(f"✅ Retrieved {len(tickers)} tickers with market cap > 0")

# 2. Get valid EODData tables that exist in the database
existing_tables = pg.get_tickers_from_EOD_tables()
valid_tickers = [t for t in tickers if t in existing_tables]

print(f"✅ {len(valid_tickers)} tickers with available EODData")

# 3. Load all 30-min data and clean
ticker_data = {}
for ticker in valid_tickers:
    table_name = f'"{ticker}_EOD_Data"'
    try:
        df = pd.read_sql_query(text(f"SELECT * FROM {table_name} WHERE open IS NOT NULL"), pg.engine)
        #checking if the length of the dataframe is greater than 0
        if df.empty:
            print(f"❌ No data found for {ticker} in {table_name}. Skipping.")
            continue
        df.reset_index(drop=True, inplace=True)
        df['date'] = pd.to_datetime(df['date'])
        df = df[df['open'].notna() & (df['open'] != 0) & df['open'].notnull()]
        df.set_index('date', inplace=True)
        df = df[['close']]  # Only keep close for correlation
        df.rename(columns={'close': ticker}, inplace=True)
        ticker_data[ticker] = df
    except Exception as e:
        print(f"⚠️ Failed to load {ticker}: {e}")

# 4. Align on common earliest date
if not ticker_data:
    print("❌ No valid data to process.")
    
print(ticker_data)

✅ Retrieved 234 tickers with market cap > 0
✅ 234 tickers with available EODData
{'AVGO':               AVGO
date              
2009-08-06    1.62
2009-08-07    1.64
2009-08-10    1.60
2009-08-11    1.57
2009-08-12    1.60
...            ...
2025-06-02  248.71
2025-06-03  256.85
2025-06-04  261.08
2025-06-05  259.93
2025-06-06  246.93

[3984 rows x 1 columns], 'MA':                 MA
date              
2006-05-25    4.60
2006-05-26    4.49
2006-05-30    4.40
2006-05-31    4.49
2006-06-01    4.75
...            ...
2025-06-02  581.22
2025-06-03  581.97
2025-06-04  584.13
2025-06-05  585.44
2025-06-06  590.12

[4789 rows x 1 columns], 'JNJ':                JNJ
date              
1970-01-02    1.22
1970-01-05    1.19
1970-01-06    1.21
1970-01-07    1.21
1970-01-08    1.22
...            ...
2025-06-02  155.40
2025-06-03  154.42
2025-06-04  153.22
2025-06-05  153.66
2025-06-06  155.03

[14003 rows x 1 columns], 'PM':                 PM
date              
2008-03-17   49.39
2008-03-18   4

##### Getting all correlation Data


In [5]:
TickerDataCopy = ticker_data.copy()
# Find common start date
# Filter out tickers where min_date is before 2010
min_dates = {ticker: df.index.min() for ticker, df in TickerDataCopy.items()}
filtered_TickerDataCopy = {ticker: df for ticker, df in TickerDataCopy.items() if min_dates[ticker] <= pd.Timestamp('2010-01-01')}
print(f"✅ Found {len(filtered_TickerDataCopy)} tickers with data starting from 2010 or later.")




if not filtered_TickerDataCopy:
    print("❌ No tickers with data starting from 2010 or later.")
TickerDataCopy = filtered_TickerDataCopy
##We go find out how many tickers have data at date 2010-01-01 specifically
#finding out how many tickers have a row with the date 2010-01-01
common_start = pd.Timestamp('2010-01-01')
min_dates = {ticker: pd.Timestamp(df.index.min()) for ticker, df in TickerDataCopy.items() if pd.Timestamp(df.index.min()) <= common_start}
if not min_dates:
    print("❌ No tickers with data starting from 2010-01-01.")
else:
    common_start = min(min_dates.values())
    TickerDataCopy = {ticker: df[df.index >= common_start] for ticker, df in TickerDataCopy.items() if df.index.min() <= common_start}
    print(f"✅ Found {len(min_dates)} tickers with data starting from 2010-01-01 or later.")
print(f"📅 Common start date for {len(min_dates)} stocks: {common_start.date()}")

# Trim all dataframes to start from common_start
for ticker in TickerDataCopy:
    TickerDataCopy[ticker] = TickerDataCopy[ticker][TickerDataCopy[ticker].index >= common_start]
    
# Print shapes of each ticker's data
for ticker, df in TickerDataCopy.items():
    print(f"📈 {ticker}: {df.shape[0]} rows from {df.index.min().date()} to {df.index.max().date()}")

# Merge all into one big DataFrame on date
merged_df = pd.concat(TickerDataCopy.values(), axis=1, join='inner').dropna()
print(f"📊 Final merged DataFrame shape: {merged_df.shape}")

# 5. Calculate correlation matrix
corr_matrix = merged_df.corr()

# 6. Rank and flatten correlations
correlation_ranks = {}
for t1, t2 in combinations(corr_matrix.columns, 2):
    corr_value = corr_matrix.loc[t1, t2]
    correlation_ranks[f"{t1}_{t2}_Corr"] = corr_value

# 7. Convert to ranked DataFrame
ranked_corrs = pd.DataFrame.from_dict(correlation_ranks, orient='index', columns=['correlation'])
ranked_corrs = ranked_corrs.sort_values(by='correlation', ascending=False)

print(ranked_corrs.head(10))

✅ Found 184 tickers with data starting from 2010 or later.
✅ Found 184 tickers with data starting from 2010-01-01 or later.
📅 Common start date for 184 stocks: 1962-01-02
📈 IBM: 15991 rows from 1962-01-02 to 2025-06-06
📈 MMM: 15990 rows from 1962-01-02 to 2025-06-06
📈 GE: 15966 rows from 1962-01-02 to 2025-06-06
📈 CAT: 15965 rows from 1962-01-02 to 2025-06-06
📈 BA: 15941 rows from 1962-01-02 to 2025-06-06
📈 KO: 15965 rows from 1962-01-02 to 2025-06-06
📈 DIS: 15965 rows from 1962-01-02 to 2025-06-06
📊 Final merged DataFrame shape: (15916, 7)
              correlation
MMM_DIS_Corr     0.938146
MMM_BA_Corr      0.915146
BA_DIS_Corr      0.912181
KO_DIS_Corr      0.902013
IBM_KO_Corr      0.898058
MMM_KO_Corr      0.897276
CAT_KO_Corr      0.865538
IBM_MMM_Corr     0.861566
BA_KO_Corr       0.840364
CAT_DIS_Corr     0.827649


##### Trading Strategy Abstract


In [6]:
class TradingStrategy():
    def __init__(self, data, window=30):
        self.rawData = data
        self.window = window
        self.workingData = pd.DataFrame()
        self.processed = False
        self.tickerResults = {}
        self.backTests = {}
        
    @abstractmethod
    def createProcessedData(self):
        """Preprocess raw data into working format"""
        pass
        
    @abstractmethod
    def calculateMetrics(self):
        """Calculate strategy-specific metrics and signals"""
        pass
        
    @abstractmethod
    def addSignalsToDataFrames(self):
        """Add generated signals to individual asset DataFrames"""
        pass
        
    @abstractmethod
    def executeBacktests(self):
        """Execute backtests for all assets"""
        pass
        
    def go(self):
        """Main execution workflow"""
        self.createProcessedData()
        self.calculateMetrics()
        self.addSignalsToDataFrames()
        self.executeBacktests()

##### Trading Strategy


In [7]:
def SIGNAL():
    return df.signal

class SignalStrategy(Strategy):
    stopLoss = 1
    takeProfit = 15
    
    
    def init(self):
        # Convert signal to proper format and register as indicator
        super().init()
        self.signal_indicator = self.I(lambda: self.data.df['signal'])
    
    def next(self):
        super().next()
        myStopLoss = self.stopLoss*0.1
        myTakeProfit = self.takeProfit*0.1
            
        try:
            # Get current signal (last value of the indicator)
            current_signal = self.signal_indicator[-1]
            
            # Execute trading logic based on signal
            if current_signal == 0 and self.position:
                self.position.close()
            elif current_signal == 1:
                if self.position.is_short:
                    self.position.close()
                if not self.position.is_long:
                    self.buy()
            elif current_signal == -1:
                if self.position.is_long:
                    self.position.close()
                if not self.position.is_short:
                    self.sell()
                    
        except IndexError:
            # Handle cases where indicators aren't fully initialized
            pass

In [8]:


class MeanReversionStrategy():
    def __init__(self, data, window = 30):
        self.rawData = data
        self.data = data
        self.workingData = pd.DataFrame()
        self.window = window  # Rolling window size for mean reversion calculations
        self.createProcessedData()
    
    def createProcessedData(self):
        """
        Preprocess the raw data to create a working DataFrame.
        This method should be called to prepare the data for the strategy.
        """
        # Rename columns to uppercase first letter for backtesting compatibility
        column_map = {
            'open': 'Open',
            'high': 'High',
            'low': 'Low',
            'close': 'Close',
            'volume': 'Volume'
        }
        
        for ticker in self.rawData.keys():
            df = self.rawData[ticker].copy()
            # Drop the symbol column if it exists
            if 'symbol' in df.columns:
                df.drop(columns=['symbol'], inplace=True)
            # Rename columns using the mapping
            df.rename(columns=column_map, inplace=True)
            # If High or Low columns don't exist, create them as a copy of Close
            if 'High' not in df.columns:
                df['High'] = df['Close']
            if 'Low' not in df.columns:
                df['Low'] = df['Close']
            self.rawData[ticker] = df
            
        self.mergeData()
        self.calculateMetrics()
        self.addSignalsToDataFrames()
        
    
        
        
    def mergeData(self):
        for ticker in self.rawData.keys():
            df = self.rawData[ticker].copy()
            # Drop the symbol column if it exists
            if 'symbol' in df.columns:
                df.drop(columns=['symbol'], inplace=True)
            # Always add prefix for consistency
            df = df.add_prefix(f"{ticker}_")
            if self.workingData.empty:
                self.workingData = df.copy()
            else:
                self.workingData = self.workingData.join(df, how='outer')
            # If {ticker}_symbol column exists, drop it
            symbol_col = f"{ticker}_symbol"
            if symbol_col in self.workingData.columns:
                self.workingData.drop(columns=[symbol_col], inplace=True)
        
        
        
    def calculateMetrics(self):
        """
        Calculate mean reversion metrics such as spread, z-score, and Bollinger Bands.
        This method is generalized to handle any number of tickers in rawData.
        """
        tickers = list(self.rawData.keys())
        n = len(tickers)
        # Calculate normalized close for each ticker
        for ticker in tickers:
            self.workingData[f'{ticker}_normalizedClose'] = (
                self.workingData[f'{ticker}_Close'] - self.workingData[f'{ticker}_Close'].rolling(window=self.window).mean()
            ) / self.workingData[f'{ticker}_Close'].rolling(window=self.window).std()

        # Calculate pairwise normalized close ratios and z-scores
        for i in range(n):
            for j in range(i + 1, n):
                t1, t2 = tickers[i], tickers[j]
                ratio_col = f'{t1}_{t2}_normalizedCloseRatio'
                mean_col = f'{t1}_{t2}_rolling_mean'
                std_col = f'{t1}_{t2}_rolling_std'
                z_col = f'{t1}_{t2}_z_score'
                # Ratio
                self.workingData[ratio_col] = (
                    self.workingData[f'{t1}_normalizedClose'] / self.workingData[f'{t2}_normalizedClose']
                )
                # Rolling mean/std
                self.workingData[mean_col] = self.workingData[ratio_col].rolling(window=self.window).mean()
                self.workingData[std_col] = self.workingData[ratio_col].rolling(window=self.window).std()
                # Z-score
                self.workingData[z_col] = (
                    self.workingData[ratio_col] - self.workingData[mean_col]
                ) / self.workingData[std_col]
                self.workingData[z_col].fillna(0, inplace=True)

                # Generate signals for both tickers based on z-score
                for ticker_signal, direction in [(t1, ('sell', 'buy')), (t2, ('buy', 'sell'))]:
                    signal_col = f'{ticker_signal}_signal'
                    if signal_col not in self.workingData.columns:
                        self.workingData[signal_col] = 'hold'
                    self.workingData.loc[self.workingData[z_col] > 1.5, signal_col] = direction[0]
                    self.workingData.loc[self.workingData[z_col] < -1.5, signal_col] = direction[1]
                    self.workingData.loc[
                        (self.workingData[z_col] >= -0.9) & (self.workingData[z_col] <= 0.9), signal_col
                    ] = 'close'
                    #shifting the signal by 1 to avoid lookahead bias
                    self.workingData[signal_col] = self.workingData[signal_col].shift(1)
        # Drop rows with NaN values in any of the calculated columns
        self.workingData.dropna(inplace=True)
        print(self.workingData.tail())
        
    def addSignalsToDataFrames(self):
        """
        Add signals to each ticker's DataFrame based on the calculated metrics.
        This method is generalized to handle any number of tickers in workingData.
        """
        tickers = list(self.rawData.keys())
        for ticker in tickers:
            signal_col = f'{ticker}_signal'
            if signal_col not in self.workingData.columns:
                self.workingData[signal_col] = 'hold'
            # add the signal to the rawData DataFrame for the appropriate ticker
            self.rawData[ticker][signal_col] = self.workingData[signal_col]
            #remove rows with NaN signals for this ticker in rawData
            self.rawData[ticker].dropna(subset=[signal_col], inplace=True)
            #replace the buy/sell/close or hold signals with 1/-1/0
            self.rawData[ticker][signal_col] = self.rawData[ticker][signal_col].replace({
                'buy': 1,
                'sell': -1,
                'close': 0,
                'hold': 0
            })
            #rename the signal column to 'signal' for consistency
            self.rawData[ticker].rename(columns={signal_col: 'signal'}, inplace=True)
            #adding the High and Low columns to the rawData which is a copy of the Close column
            
            
    def go(self):
        #create self.tickerResults which is a dictionary
        self.tickerResults = {}
        self.backTests = {}
        for ticker in self.rawData.keys():
            print(f"Running backtest for {ticker}...")
            df = self.rawData[ticker].copy()
            print(df.columns)
            
            
            
            bt = Backtest(
                df,
                SignalStrategy,
                cash=10_000,
                commission=.002
            )
            
            stats, heatmap = bt.optimize(
                stopLoss = range(1, 5, 1),
                takeProfit = range(12, 20, 1),
                maximize='Sharpe Ratio',
                return_heatmap=True,
                max_tries= 50,
            )
            self.backTests[ticker] = bt
            self.tickerResults[ticker] = stats
        
        
        
        
    
            
        
        
        
        

In [9]:
ticker1 = 'MMM'
ticker2 = 'BA'

# Fetching EOD data for the two tickers
df1 = pg.getTickerEODData(ticker1)
df2 = pg.getTickerEODData(ticker2)

# Only keep data after 2010 (date is the index)
df1 = df1[df1.index >= pd.Timestamp('2010-01-01')]
df2 = df2[df2.index >= pd.Timestamp('2010-01-01')]

# Initialize and run the Mean Reversion Strategy
data_dict={ticker1: df1, ticker2: df2}

mrs = MeanReversionStrategy(data_dict, window=30)
mrs.go()



✅ Retrieved 15990 rows of EOD data for MMM
✅ Retrieved 15941 rows of EOD data for BA
            MMM_Open  MMM_Close  MMM_Volume  MMM_High  MMM_Low  BA_Open  \
date                                                                      
2025-06-02    147.87     146.40     2825788    146.40   146.40   210.98   
2025-06-03    146.31     148.13     2343553    148.13   148.13   212.00   
2025-06-04    148.91     146.81     2347013    146.81   146.81   214.77   
2025-06-05    147.04     146.26     3568399    146.26   146.26   212.50   
2025-06-06    148.00     145.50     2297132    145.50   145.50   210.24   

            BA_Close  BA_Volume  BA_High  BA_Low  MMM_normalizedClose  \
date                                                                    
2025-06-02    211.47    7561816   211.47  211.47             0.357026   
2025-06-03    213.43    8106100   213.43  213.43             0.578184   
2025-06-04    211.98    5705300   211.98  211.98             0.306097   
2025-06-05    209.02    

/tmp/ipykernel_40001/696649233.py:95: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  self.workingData[z_col].fillna(0, inplace=True)
/tmp/ipykernel_40001/696649233.py:128: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.rawData[ticker][signal_col] = self.rawData[ticker][signal_col].replace({
/tmp/

Running backtest for BA...
Index(['Open', 'Close', 'Volume', 'High', 'Low', 'signal'], dtype='object')


##### Getting Correlation Basket


In [18]:
tickerDict = ticker_data.copy()
tickerDict.keys()

closePriceDf = pd.DataFrame()

# Create a DataFrame with only the 'close' prices for each ticker
for ticker, df in tickerDict.items():
    closePriceDf[ticker] = df[ticker]

# Ensure the DataFrame is sorted by date
closePriceDf.sort_index(inplace=True)
#dropping Nan values


/tmp/ipykernel_40001/468407267.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  closePriceDf[ticker] = df[ticker]
/tmp/ipykernel_40001/468407267.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  closePriceDf[ticker] = df[ticker]
/tmp/ipykernel_40001/468407267.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = 

In [27]:
#where the index is greater than or equal to 2010-01-01
closePriceDf = closePriceDf[closePriceDf.index >= pd.Timestamp('2010-01-01')]
#removing the columns with more than 20% NaN values
closePriceDf = closePriceDf.dropna(thresh=len(closePriceDf) * 0.8, axis=1)
# Display the first few rows of the close price DataFrame
print(closePriceDf.head())

             AVGO      MA     JNJ     PM     IBM     NOW    TBB    RTX  \
date                                                                     
2020-01-02  32.24  303.39  145.97  85.19  129.35  291.24  26.97  90.32   
2020-01-03  31.42  300.43  144.28  85.03  128.32  291.10  27.01  90.45   
2020-01-06  31.37  301.23  144.10  86.02  128.09  292.87  27.01  90.64   
2020-01-07  31.26  300.21  144.98  86.40  128.17  292.91  27.03  90.35   
2020-01-08  30.87  305.10  144.96  88.04  129.24  295.65  27.11  90.49   

              ADBE   SCHW  ...     PSX    SLB     URI     LHX     GM       EW  \
date                       ...                                                  
2020-01-02  334.43  48.23  ...  112.20  40.17  168.65  203.31  37.38  77.5000   
2020-01-03  331.81  47.01  ...  108.49  40.56  165.71  210.47  36.32  77.2100   
2020-01-06  333.71  47.34  ...  107.81  40.82  165.38  211.72  35.84  78.2667   
2020-01-07  333.39  47.62  ...  108.14  40.61  165.08  212.73  35.15  77.670

In [36]:
def find_correlated_group(close_prices, correlation_threshold=0.7):
    """
    Identifies a group of highly correlated stocks from close price data.
    
    Parameters:
    close_prices (pd.DataFrame): DataFrame with tickers as columns and dates as index
    correlation_threshold (float): Minimum correlation coefficient to consider stocks as correlated
    
    Returns:
    list: Tickers in the largest correlated group
    """
    # 1. Calculate correlation matrix
    corr_matrix = close_prices.corr()
    
    # 2. Convert correlation to distance matrix (0 = perfect correlation, 2 = no correlation)
    distance_matrix = np.sqrt(2 * (1 - corr_matrix))
    
    # 3. Perform hierarchical clustering
    linkage_matrix = linkage(distance_matrix, 'average')
    
    # 4. Cluster based on correlation threshold
    distance_threshold = np.sqrt(2 * (1 - correlation_threshold))
    clusters = fcluster(linkage_matrix, distance_threshold, criterion='distance')
    
    # 5. Find largest cluster
    cluster_labels, counts = np.unique(clusters, return_counts=True)
    largest_cluster_id = cluster_labels[np.argmax(counts)]
    
    # 6. Get tickers in the largest cluster
    correlated_tickers = close_prices.columns[clusters == largest_cluster_id].tolist()
    
    return correlated_tickers

In [39]:
correlated_group = find_correlated_group(closePriceDf, correlation_threshold=0.8)
correlated_group

/tmp/ipykernel_40001/738397474.py:19: ClusterWarning: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  linkage_matrix = linkage(distance_matrix, 'average')


['BSX', 'HWM']